### Comprehension Burden for Sequencing Documents

In [348]:
import requests
from collections import Counter

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import networkx as nx

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [349]:
'''
Knowledge Graph
'''
kg_path = "../graph_query/graphs/knowledge_graph.gpickle"
kg = nx.read_gpickle(kg_path)
kg_labels = [str(x) for x in list(kg.nodes())[1:]]
n_labels = len(kg_labels)

In [350]:
'''
Get content from a given set of URLs.
'''
def get_content():
    f = open('sample_urls.txt', 'r')
    l = f.readlines()
    docs = {}
    index = {}
    
    counter = 0
    for url in l:
        docs[url] = requests.get(url).content
        index[url] = counter
        counter += 1
    return docs, index

### Calculating Relationship Score: S(i, j)

In [351]:
'''
Term Frequency Array for a particular document.
'''
def get_tfd(content):
    word_count_dict = Counter(w for w in kg_labels 
                              if w.lower() in content.lower())
    common = word_count_dict.most_common()
    
    frequency_arr = [0]*len(kg_labels)
    
    for common_word in common:
        common_word_index = kg_labels.index(common_word[0])
        frequency_arr[common_word_index] = common_word[1]
    return frequency_arr

In [352]:
content, index = get_content()

In [353]:
'''
Building word_data a document (rows) by term frequency (columns) matrix.
'''
tfd_data = {}
for url, cont in content.items():
    tfd_data[url] = get_tfd(cont)

tfd_arr = []
for key in index.keys():
    tfd_arr.append(key.replace("\n", ""))

word_data = {'TFD':tfd_arr}

for label in kg_labels:
    word_data[label] = [None]*len(index)

for url, words_in_doc in tfd_data.items():
    url_index = index[url]
    for i in range(0, n_labels, 1):
        word = kg_labels[i]
        word_data[word][url_index] = words_in_doc[i]

In [354]:
'''
(DTF)^T(DTF) = Coocurence Matrix
'''
document_term_frequency = pd.DataFrame(word_data).set_index('TFD')
dtf_asint = document_term_frequency.astype(int)
coocc = dtf_asint.T.dot(dtf_asint)

### Significance of a concept in a document: \lambda(c, i)

In [355]:
def get_significance_score(concept, document):
    concept_index = document_term_frequency.columns.get_loc(concept)
    freq = dtf_asint.iloc[index[document]][concept_index]
    coocc_row = coocc.iloc[concept_index,:] 
    r = np.array(coocc_row)
    return freq+0.3*np.count_nonzero(r)

### Key Sections

In [357]:
key_doc = {}

for each_concept in kg_labels:
    key_doc[each_concept] = None
    key_max = 0
    for each_document in content.keys():
        s = get_significance_score(each_concept, each_document)
        if(s > key_max):
            key_max = s
            key_doc[each_concept] = each_document

# for key, val in key_doc.items():
#     if(val != None):
#         print(key, val)

### Comprehension Burden